In [1]:
from nlp_jzar import *
import pandas as pd

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

# Find the best params for the model

In [3]:
best_performance(train_df,test_df)

/home/jzar/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1261, in fbe

/home/jzar/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1261, in fbe

/home/jzar/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "/home/jzar/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 1261, in fbe

KeyboardInterrupt: 

# Apply them

In [4]:
clf = use_model(train_df, 
                test_df, 
                mix_texts=False, 
                clean_texts=True, 
                vectorization="tfidf", 
                use_LSA=False)

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


,text,is_disaster
3070,Terrifying POV footage captures violent landin...,True
2324,Police: Tennessee theater shooting suspect was...,True
594,Three-alarm fire burning in Antioch apartment ...,True
859,SEVERE WEATHER BULLETIN No. 6\nFOR: TYPHOON Û...,True
2117,Byproduct of #metal price meltdown is a higher...,True


,text,is_disaster
1345,Do you remember when I suddenly electrocute yo...,False
674,@_LalaDeviluke - consequences could have been ...,False
886,@TheElmagoo @GOPTeens @FoxNews @pattonoswalt a...,False
2641,@spookyerica sleeping with sirens?,False
2234,'I must not fear. Fear is the mind-killer. Fea...,False
